In [ ]:
# Importera nödvändiga bibliotek
# Pandas och numpy används för datahantering, matplotlib och seaborn för visualisering
# sklearn innehåller maskininlärningsmodeller och verktyg för uppdelning och normalisering
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

# Ladda datasetet "Travel Reviews"
# Datasetet laddas från en CSV-fil som innehåller recensioner för olika resedestinationer
# Denna data används för att träna och utvärdera k-NN-modellen
dataset = pd.read_csv('tripadvisor_review.csv')

# Utforska datasetet
# Visa de första raderna i datasetet för att få en översikt över strukturen och innehållet
# Detta är viktigt för att förstå vilka kolumner och typer av data vi arbetar med
dataset.head()

In [ ]:
# Separera features (X) och målvariabel (y)
# X innehåller alla features som används för att förutsäga målvariabeln
# Vi tar bort kolumnen 'User ID' eftersom den inte bidrar till klassificeringsmodellen
# y är målvariabeln, dvs. den sista kolumnen, som representerar recensionens kategori (betyg från 0 till 4)
X = dataset.drop(columns=['User ID']).iloc[:, :-1]  # Alla kolumner utom den sista och 'User ID' används som features
y = dataset.iloc[:, -1].round().astype('int').astype('category')  # Sista kolumnen som målvariabel, avrundad och konverterad till kategorisk

# Definiera uppdelningar för tränings- och testdata
# Vi använder tre olika proportioner mellan tränings- och testdata för att analysera hur detta påverkar modellens prestanda
train_test_splits = [(0.75, 0.25), (2/3, 1/3), (0.5, 0.5)]
k_values = [3, 7, 15]  # Tre olika värden på k för att undersöka hur valet av k påverkar modellens prestanda
results = []

# Normalisera data för användning i k-NN
# Vi använder MinMaxScaler för att skala features till intervallet [0, 1]
# Detta hjälper k-NN att arbeta effektivare med features som kan ha olika skalor
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)

In [ ]:
# k-NN med originaldata
for k in k_values:
    print(f"\n--- Körning för k = {k} ---")
    for train_size, test_size in train_test_splits:
        accuracies = []
        best_cm = None
        best_acc = 0
        for iteration in range(100):
            # Dela upp datasetet i tränings- och testdata
            # random_state=None gör att varje uppdelning blir slumpmässig för varje iteration
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, test_size=test_size, random_state=None)
            
            # Skapa och träna k-NN klassificeraren
            # Vi skapar en k-NN klassificerare med ett specifikt värde på k
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)
            
            # Gör prediktioner på testdatan
            # Prediktionerna används för att utvärdera hur bra modellen har tränat
            y_pred = knn.predict(X_test)
            
            # Beräkna accuracy och confusion matrix
            # Accuracy mäter hur många korrekta förutsägelser modellen gjorde
            # Confusion matrix hjälper oss att förstå hur förutsägelserna fördelades mellan klasserna
            acc = accuracy_score(y_test, y_pred)
            cm = confusion_matrix(y_test, y_pred)
            accuracies.append(acc)
            
            # Spara den bästa confusion matrix för visning
            if acc > best_acc:
                best_acc = acc
                best_cm = cm
        
        # Beräkna medelvärdet av accuracy över 100 iterationer
        # Medelvärdet ger en mer pålitlig bild av modellens prestanda över flera uppdelningar och körningar
        avg_accuracy = np.mean(accuracies)
        
        # Lägg till resultat för visualisering och sammanställning
        results.append((f'k-NN (k={k})', train_size, test_size, avg_accuracy, best_cm))
        
        # Skriv ut medelvärdet av accuracy för varje uppdelning
        print(f'k-NN (k={k}) - Train size = {train_size:.2f}, Test size = {test_size:.2f}, Average Accuracy: {avg_accuracy:.2f}')

In [ ]:
# k-NN med normaliserat data
for k in k_values:
    print(f"\n--- Körning för k = {k} ---")
    for train_size, test_size in train_test_splits:
        accuracies = []
        best_cm = None
        best_acc = 0
        for iteration in range(100):
            # Dela upp datasetet i tränings- och testdata
            # Använd normaliserade features för att undersöka om detta förbättrar modellens prestanda
            X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, train_size=train_size, test_size=test_size, random_state=None)
            
            # Skapa och träna k-NN klassificeraren
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)
            
            # Gör prediktioner på testdatan
            y_pred = knn.predict(X_test)
            
            # Beräkna accuracy och confusion matrix
            acc = accuracy_score(y_test, y_pred)
            cm = confusion_matrix(y_test, y_pred)
            accuracies.append(acc)
            
            # Spara den bästa confusion matrix för visning
            if acc > best_acc:
                best_acc = acc
                best_cm = cm
        
        # Beräkna medelvärdet av accuracy över 100 iterationer
        avg_accuracy = np.mean(accuracies)
        
        # Lägg till resultat för visualisering och sammanställning
        results.append((f'k-NN (k={k}) (Normalized)', train_size, test_size, avg_accuracy, best_cm))
        
        # Skriv ut medelvärdet av accuracy för varje uppdelning
        print(f'k-NN (k={k}) (Normaliserat) - Train size = {train_size:.2f}, Test size = {test_size:.2f}, Average Accuracy: {avg_accuracy:.2f}')

In [ ]:
# Sammanfattning av bästa resultat
# Sortera resultaten och visa de tre bästa exekveringarna baserat på högsta accuracy
best_results = sorted(results, key=lambda x: x[3], reverse=True)[:3]
for model, train_size, test_size, acc, cm in best_results:
    print(f'Model = {model}, Train size = {train_size:.2f}, Test size = {test_size:.2f}, Accuracy: {acc:.2f}')
    # Visa confusion matrix för de bästa exekveringarna för att förstå modellens prestanda mer i detalj
    sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
    plt.title(f'Confusion Matrix for {model} (Train={train_size:.2f}, Test={test_size:.2f})')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()